In [27]:
import pandas as pd
from sa_package.my_selenium import get_driver

# Selenium Driver 불러오기

In [28]:
driver = get_driver()

In [29]:
driver.get("https://game.naver.com/ranking/top/daily")

# 페이지 내용 읽기

In [31]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup)
#root > div > div.ranking_container__2XBZD > div > div.ranking_list_wrap__3E2TP > ol:nth-child(1) > li:nth-child(1) > div > button > span.lounge_item_info__3kf7q > span.lounge_item_name_wrap__1C2mF > strong

In [32]:
game_list = soup.select("#root > div > div.ranking_container__2XBZD > div > div.ranking_list_wrap__3E2TP > ol > li")
print(len(game_list))
print(game_list[0])


100
<li class="ranking_list_item__2kpJr"><div class="lounge_item_wrap__1tfod"><button class="lounge_item_lounge__1VWSu" type="button"><span class="lounge_item_rank_area__9CMsX"><strong class="lounge_item_rank__2O7_v">1</strong><span class="lounge_item_rank_status__3x80u lounge_item_equal__21vne"></span></span><span class="lounge_item_thumbnail_area__1aAHo" style='background-image: url("https://nng-phinf.pstatic.net/MjAyMTA5MDhfMTk3/MDAxNjMxMDY4NDk3MzYx.xJYcVzRN9rJhjo_Wk3glFkqgEEQYbCAxnd49BK_L5dwg.0drLMwYFCWTAYc7e0Cy0zQUEWC-kFXEM79k__1uN7Ukg.PNG/League_of_Legends.png"), url("https://nng-phinf.pstatic.net/MjAyMTA0MzBfNjAg/MDAxNjE5NzczMzI0OTEx.yZhA1c6Slf68dURtJLkXZFC7RDx-jFCR9zrQcXNgWx0g.XhMJO6ugRer_fgHuKUgh9imNUp1FjyESWxYWNlOUuTMg.PNG/150-x-1502x.png");'></span><span class="lounge_item_info__3kf7q"><span class="lounge_item_name_wrap__1C2mF"><strong class="lounge_item_name__2SBpA">리그 오브 레전드</strong></span><span class="lounge_item_sub__29Ye0">MOBA</span></span></button><div class="lounge_i

In [37]:
game_name = game_list[0].select(".lounge_item_name__2SBpA")
print(game_name)
# print(game_name[0].text)
# print(game_name.text)
for i in range(len(game_name)):
    print(game_name[i].text)


[<strong class="lounge_item_name__2SBpA">리그 오브 레전드</strong>]
리그 오브 레전드


In [45]:
game_rank_status = game_list[0].select(".lounge_item_rank_status__3x80u")[0]
print(game_rank_status)
print(game_rank_status.get_attribute_list("class")[1])
print(game_rank_status.get_attribute_list("class")[1].split("_")[2])

<span class="lounge_item_rank_status__3x80u lounge_item_equal__21vne"></span>
lounge_item_equal__21vne
equal


# 읽은 내용 정리해서 스프레드시트에 올리기

In [52]:
rank_list = []
change_list = []
name_list = []
genre_list = []

for i in range(len(game_list)):
    # 순위
    game_rank = game_list[i].select(".lounge_item_rank_area__9CMsX > strong")[0].text
    
    # 변동
    game_rank_status = game_list[i].select(".lounge_item_rank_status__3x80u")[0].get_attribute_list("class")[1].split("_")[2]
    if game_rank_status == "equal":
        game_rank_change = 0
    elif game_rank_status == "up":
        game_rank_change = int(game_list[i].select(".lounge_item_rank_status__3x80u")[0].text)
    else:
        game_rank_change = (-1) * int(game_list[i].select(".lounge_item_rank_status__3x80u")[0].text)
    
    # 게임이름
    game_name = game_list[i].select(".lounge_item_name__2SBpA")[0].text

    # 게임장르
    game_genre = game_list[i].select(".lounge_item_sub__29Ye0")[0].text
    
    rank_list.append(game_rank)
    change_list.append(game_rank_change)
    name_list.append(game_name)
    genre_list.append(game_genre)
    # print(game_rank, game_rank_change, game_name, game_genre)

game_df = pd.DataFrame(
    data={
        "순위":rank_list,
        "순위변동":change_list,
        "게임명":name_list,
        "장르":genre_list
    }
)


In [53]:
game_df.head()

,순위,순위변동,게임명,장르
0,1,0,리그 오브 레전드,MOBA
1,2,0,트리 오브 세이비어 M,MMORPG
2,3,8,승리의 여신: 니케,FPS/TPS
3,4,-1,네이버 게임 오리지널 시리즈,독점 콘텐츠를 즐겨보세요
4,5,-1,PUBG: 배틀그라운드,서바이벌 슈팅


In [54]:
import datetime
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

from sa_package.my_gspread import get_df_from_gspread, write_df_to_sh, create_worksheet


SCOPE = ["https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"]
CREDS = ServiceAccountCredentials.from_json_keyfile_name("C:/Users/SANDBOX/sandbox_workspace/data_management/dm_workshop/coding_day/key.json", SCOPE)
SH_ID = "1l1LhuVXF5WmRX2V-LnJArAYN3Gc_4mMSWKcCO---Sxg"

create_worksheet(
    sh_id=SH_ID,
    worksheet_name=f"게임장르_승아_{datetime.date.today().strftime('%Y%m%d')}",
    creds=CREDS
)
write_df_to_sh(
    sh_id=SH_ID,
    worksheet_name=f"게임장르_승아_{datetime.date.today().strftime('%Y%m%d')}",
    df=game_df,
    include_header=True,
    creds=CREDS
)

<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1l1LhuVXF5WmRX2V-LnJArAYN3Gc_4mMSWKcCO---Sxg/values/%EA%B2%8C%EC%9E%84%EC%9E%A5%EB%A5%B4_%EC%8A%B9%EC%95%84_20221104%21A1?valueInputOption=USER_ENTERED&alt=json returned "Unable to parse range: 게임장르_승아_20221104!A1". Details: "Unable to parse range: 게임장르_승아_20221104!A1">
